In [99]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output #this allows us to do callbacks
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import os
import pyodbc

#debugging
import time
start_time = time.time()

print ('Import complete. This took', time.time() - start_time, 'to run. ')  #B/C fku vscode for being slow.

Import complete. This took 0.0 to run. 


In [100]:
#Establish Connection Function

def get_conn():
    server = "DC1QEDCISQLEEV;" if os.path.exists("Q:") else "VDBEDCISANDBOX;"
    odbc_conn = "DRIVER={SQL Server};SERVER=" + server
    return pyodbc.connect(odbc_conn, autocommit=True) #pyodbc.connect(odbc_conn)

print('Connection function Complete.')

Connection function Complete.


In [101]:
#Get data

#df that has top 10 companies at FRB based on total dep
df_top_cmp_by_dep = '''
SELECT TOP 10
COMPANY_NM
,SUM(RDFI_AMT) as Total_Deposits
,AVG(RDFI_AMT) AS AVG_DEPOSIT
FROM [DataScienceAndAnalytics].[stage].[PAYROLL_TRANSACTIONS]
WHERE 1 = 1
AND PROD_DT >= '2019-12-01' 
AND COMPANY_NM <> 'First Republic'
GROUP BY COMPANY_NM
order by 2 desc 
'''

df_top_cmp_by_dep = pd.read_sql(df_top_cmp_by_dep, get_conn())

df_top_cmp_by_dep.head()


,COMPANY_NM,Total_Deposits,AVG_DEPOSIT
0,DODGE & COX,1.158034e+08,1.301162e+06
1,PIMCO,1.532909e+07,2.129040e+05
2,BAIN CAPITAL LP,1.175934e+07,8.459956e+04
3,RBC CAPITAL MARK,6.976610e+06,1.245823e+05
4,TRINET,6.862659e+06,6.313394e+03


In [102]:
 #Create bolean mask for df, this is so we can create the callbacks in dash.
# inputCompany = 'PIMCO'

# mask = df_top_cmp_by_dep['COMPANY_NM'].str.contains(inputCompany)

# masked_df = df_top_cmp_by_dep[mask]

# df_top_cmp_by_dep[mask].head()


In [103]:
#Establish Graph Data with plotly. This is done with traces.

# Top10Trace = go.Bar(x=df_top_cmp_by_dep['COMPANY_NM']
#                   , y=df_top_cmp_by_dep['Total_Deposits']
#                   , name='Total Deposits')

# #Create a data variable so this can be easily switched later. 
# data = [Top10Trace]

# #Create a layout that can be easily switched later 
# layout = dict(Title =  ('Total Deposits for ',inputCompany)  
#              ,showlegend = False)

#Set Figure
fig = dict(data = data
           ,layout = layout)

In [0]:
#Dash Starts here
app = dash.Dash()

app.layout = html.Div([



    html.Div(html.H1(children="Transaction Dash - Test"))
,
    html.Label('Transaction Graph')
,
   html.Div(  
    dcc.Input(
         id='company-input'
        ,placeholder='Enter Company Name'
        ,type='text'
        ,value='PIMCO'
             )
            )
,
    html.Div(
        dcc.Graph(id = 'trans_deposit_graph'
                  ,figure = fig
                 )
            )
,
    html.Div(
        dcc.Dropdown(
            options = [
            {'label': 'Label 1 Here', 'value': 'Value 1'}
           ,{'label': 'Label 2 Here', 'value': 'Value 2'}
           ,


                     ]
                    )
            )

])

#Below allows us to set user input functionality with call backs.
#Output = where the user input will be reflected. e.g. a graph ('name-of-output','what-youre-updating')
#Input = the place where the user gets to choose the option ('what you're referencing, 'value from that input' )
@app.callback(dash.dependencies.Output('trans_deposit_graph',"figure")
              ,[dash.dependencies.Input('company-input','value')]
              )
def update_fig(input_value):
    #Create bolean mask for df, this is so we can create the callbacks in dash.
    mask = df_top_cmp_by_dep['COMPANY_NM'].str.contains(input_value)

    masked_df = df_top_cmp_by_dep[mask]

    Top10Trace = go.Bar(x=masked_df['COMPANY_NM']
                      , y=masked_df['Total_Deposits']
                      , name='Total Deposits')

    data = []

    data.append(Top10Trace)

    layout = {'title':'Callback Graph'}

    return {
        "data": data
        ,"layout": layout
    }





In [0]:
if __name__ == '__main__':
    app.run_server(debug=False)